In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import json
import os
from PIL import Image
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 12
NUM_EPOCHS = 50
NUM_WORKERS = 2
IMAGE_HEIGHT = 288
IMAGE_WIDTH = 512
PIN_MEMORY = True

TUSIMPLE_ROOT = "/kaggle/input/masked-dataset"
ANNOTATION_FILE = "/kaggle/input/masked-dataset/processed/annotations.json"
CHECKPOINT_PATH = "my_tusimple_attention_model.pth.tar"

print(f"Using device: {DEVICE}")

Using device: cuda


In [ ]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
])

In [4]:
class LaneDataset(Dataset):
    def __init__(self, annotations, root_dir, transform=None):
        self.annotations = annotations
        self.root_dir = root_dir
        self.transform = transform if transform is not None else transforms.ToTensor()

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.annotations[idx]['image'])
        mask_path = os.path.join(self.root_dir, self.annotations[idx]['mask'])
        
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        image_tensor = self.transform(image)
        mask_tensor = transforms.ToTensor()(mask)
        
        return image_tensor, (mask_tensor > 0).float()


In [ ]:
# 3Model Architecture with Attention
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels), nn.ReLU(inplace=True)
        )
    def forward(self, x): return self.double_conv(x)

class AttentionGate(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionGate, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

class AttentionUNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(AttentionUNET, self).__init__()
        self.inc = DoubleConv(in_channels, 64)
        self.down1 = nn.MaxPool2d(2); self.conv1 = DoubleConv(64, 128)
        self.down2 = nn.MaxPool2d(2); self.conv2 = DoubleConv(128, 256)
        self.down3 = nn.MaxPool2d(2); self.conv3 = DoubleConv(256, 512)
        self.down4 = nn.MaxPool2d(2); self.bottleneck = DoubleConv(512, 1024)
        
        self.up1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.Att1 = AttentionGate(F_g=512, F_l=512, F_int=256)
        self.up_conv1 = DoubleConv(1024, 512)
        
        self.up2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.Att2 = AttentionGate(F_g=256, F_l=256, F_int=128)
        self.up_conv2 = DoubleConv(512, 256)
        
        self.up3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.Att3 = AttentionGate(F_g=128, F_l=128, F_int=64)
        self.up_conv3 = DoubleConv(256, 128)
        
        self.up4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.Att4 = AttentionGate(F_g=64, F_l=64, F_int=32)
        self.up_conv4 = DoubleConv(128, 64)
        
        self.outc = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1); x2 = self.conv1(x2)
        x3 = self.down2(x2); x3 = self.conv2(x3)
        x4 = self.down3(x3); x4 = self.conv3(x4)
        x5 = self.down4(x4); x5 = self.bottleneck(x5)
        
        up1 = self.up1(x5)
        att_x4 = self.Att1(g=up1, x=x4) 
        concat1 = torch.cat([up1, att_x4], dim=1)
        up1_conv = self.up_conv1(concat1)
        
        up2 = self.up2(up1_conv)
        att_x3 = self.Att2(g=up2, x=x3) 
        concat2 = torch.cat([up2, att_x3], dim=1)
        up2_conv = self.up_conv2(concat2)
        
        up3 = self.up3(up2_conv)
        att_x2 = self.Att3(g=up3, x=x2) 
        concat3 = torch.cat([up3, att_x2], dim=1)
        up3_conv = self.up_conv3(concat3)
        
        up4 = self.up4(up3_conv)
        att_x1 = self.Att4(g=up4, x=x1) 
        concat4 = torch.cat([up4, att_x1], dim=1)
        up4_conv = self.up_conv4(concat4)
        
        return self.outc(up4_conv)


In [ ]:
# Training and Evaluation Functions
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().to(device=DEVICE)
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        loop.set_postfix(loss=loss.item())

def check_accuracy(loader, model, device="cuda"):
    dice_score = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            dice_score += (2 * (preds * y).sum()) / ((preds + y).sum() + 1e-8)
    avg_dice_score = dice_score / len(loader)
    model.train()
    return avg_dice_score


In [ ]:
def main():
    print("📂 Loading annotations...")
    with open(ANNOTATION_FILE, "r") as f:
        all_annotations = json.load(f)
    
    train_ann, val_ann = train_test_split(all_annotations, test_size=0.2, random_state=42)

    train_dataset = LaneDataset(annotations=train_ann, root_dir=TUSIMPLE_ROOT, transform=train_transform)
    val_dataset = LaneDataset(annotations=val_ann, root_dir=TUSIMPLE_ROOT, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=False)

    model = AttentionUNET(in_channels=3, out_channels=1).to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scaler = torch.amp.GradScaler('cuda') 
    
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3, min_lr=1e-6) 
    
    best_dice_score = -1.0
    start_epoch = 0 

    if os.path.exists(CHECKPOINT_PATH):
        print(f"🔄 Checkpoint found! Resuming training from {CHECKPOINT_PATH}")
        checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
        model.load_state_dict(checkpoint["state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer"])
        best_dice_score = checkpoint.get("best_dice", -1.0)
        start_epoch = checkpoint.get("epoch", 0)
        print("✅ Model and optimizer states loaded.")
        
    print(f"\n--- Starting Training from Epoch {start_epoch + 1} ---")
    for epoch in range(start_epoch, NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler)
        
        current_dice = check_accuracy(val_loader, model, device=DEVICE)
        
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] | Val Dice: {current_dice:.4f} | Best: {best_dice_score:.4f} | LR: {optimizer.param_groups[0]['lr']:.6f}")
        
        if current_dice > best_dice_score:
            best_dice_score = current_dice
            print(f"✅ New best model! Saving to {CHECKPOINT_PATH}")
            checkpoint = {
                "state_dict": model.state_dict(),
                "optimizer": optimizer.state_dict(),
                "best_dice": best_dice_score,
                "epoch": epoch
            }
            torch.save(checkpoint, CHECKPOINT_PATH)
        
        scheduler.step(current_dice)

In [8]:
if __name__ == "__main__":
    main()

📂 Loading annotations...

--- Starting Training from Epoch 1 ---


  0%|          | 0/242 [00:00<?, ?it/s]/tmp/ipykernel_36/1391012976.py:8: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 242/242 [03:23<00:00,  1.19it/s, loss=0.255]


Epoch [1/50] | Val Dice: 0.3504 | Best: -1.0000 | LR: 0.000100
✅ New best model! Saving to my_tusimple_attention_model.pth.tar


100%|██████████| 242/242 [03:22<00:00,  1.20it/s, loss=0.193]


Epoch [2/50] | Val Dice: 0.5342 | Best: 0.3504 | LR: 0.000100
✅ New best model! Saving to my_tusimple_attention_model.pth.tar


...

Epoch [48/50] | Val Dice: 0.7606 | Best: 0.7634 | LR: 0.000003


100%|██████████| 242/242 [03:22<00:00,  1.20it/s, loss=0.0134]


Epoch [49/50] | Val Dice: 0.7602 | Best: 0.7634 | LR: 0.000003


100%|██████████| 242/242 [03:22<00:00,  1.20it/s, loss=0.0133]


Epoch [50/50] | Val Dice: 0.7601 | Best: 0.7634 | LR: 0.000003


In [9]:
print("Done")

Done
